In [1]:
#import importlib
#importlib.reload(_prepare)

In [2]:
import warnings
warnings.filterwarnings(action='once')


Original idea from: https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/


In this post, we discuss techniques to visualize the output and results from topic model (LDA) based on the gensim package. I will be using a portion of the 20 Newsgroups dataset since the focus is more on approaches to visualizing the results.

Let’s begin by importing the packages and the 20 News Groups dataset.

In [3]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['linkremoved',' <link removed>','usernameremoved','<usernameremoved>','<linkremoved>','usernameremoved_usernameremoved','linkremoved_linkremoved'])


%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

c:\users\gonza\tesisenv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
c:\users\gonza\tesisenv\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
c:\users\gonza\tesisenv\lib\site-packages\thinc\neural\_custom_kernels.py:36: ResourceWarning: unclosed file <_io.TextIOWrapper name='c:\\users\\gonza\\tesisenv\\lib\\site-packages\\thinc\\neural\\_custom_kernels.cu' mode='r' encoding='utf8'>
  SRC = (PWD / "_custom_kernels.cu").open("r", encoding="utf8").read()
c:\users\gonza\tesisenv\lib\site-packages\thinc\neural\_custom_kernels.py:39: ResourceWarn

In [4]:
#Instructions about how to install mallet are available here: http://mallet.cs.umass.edu/download.php

'''
Windows installation: After unzipping MALLET, set the environment variable %MALLET_HOME% to point to the MALLET directory.
In all command line examples, substitute bin\mallet for bin/mallet.
'''
import os
from gensim.models.wrappers import LdaMallet


path_to_mallet_binary = "C:\\mallet-2.0.8\\bin\\mallet"
os.environ.update({'MALLET_HOME':r'C:\mallet-2.0.8'}) #OJO!, por alguna razon mallet solo puede estar disponible en esa carpeta


## Import Cambridge Analytica datasets


In [5]:
#this libraries are for this dataset
import pandas as pd


In [6]:
file_name_1 = 'english_europe_tweets_20190411.csv'
file_name_2 = 'english_northamerica_tweets_20190411.csv'


In [7]:
df_1  = pd.read_csv('data/cambridge_analytica/regional_datasets/'+file_name_1)
df_2  = pd.read_csv('data/cambridge_analytica/regional_datasets/'+file_name_2)

In [8]:
#df_1 = df_1.sample(1000)  #remove this line
#df_2 = df_2.sample(1000)  #remove this line

In [9]:
df_1.head()

,Unnamed: 0,created_at2,id,created_at,user.screen_name,user_location,coordinates,text,full_text,retweet_created_at,retweet_full_text,texto_completo,ubicacion_encontrada
0,2,2018-04-01,id-980480940567924736,Sun Apr 01 16:24:07 +0000 2018,CamHatch,"Rotterdam, The Netherlands",NaN,Meet our newest product: the transparent CamHa...,Meet our newest product: the transparent CamHa...,NaN,NaN,Meet our newest product: the transparent CamHa...,Netherlands
1,17,2018-04-01,id-980481203110281216,Sun Apr 01 16:25:09 +0000 2018,iky86,stoke on trent,NaN,@SupportOurLefty This week has been the most #...,@SupportOurLefty This week has been the most #...,NaN,NaN,<usernameremoved> This week has been the most ...,United Kingdom
2,43,2018-04-01,id-980480454846566401,Sun Apr 01 16:22:11 +0000 2018,MichaelJF80,"Liverpool, England",NaN,Pulitzer prize for this lady. Investigative jo...,NaN,NaN,NaN,Pulitzer prize for this lady. Investigative jo...,United Kingdom
3,45,2018-04-01,id-980480424429400071,Sun Apr 01 16:22:04 +0000 2018,ruechids1,"Enfield, London",NaN,"Theres no more secrecy, confident and privacy ...","Theres no more secrecy, confident and privacy ...",NaN,NaN,"Theres no more secrecy, confident and privacy ...",United Kingdom
4,51,2018-04-01,id-980480325083164673,Sun Apr 01 16:21:40 +0000 2018,Hugh_Macfarlane,"Glenrothes, Fife",NaN,AggregateIQ: the obscure Canadian tech firm an...,NaN,NaN,NaN,AggregateIQ: the obscure Canadian tech firm an...,United Kingdom


In [10]:
df_2.head()

,Unnamed: 0,created_at2,id,created_at,user.screen_name,user_location,coordinates,text,full_text,retweet_created_at,retweet_full_text,texto_completo,ubicacion_encontrada
0,1,2018-04-01,id-980480936205783047,Sun Apr 01 16:24:06 +0000 2018,stanscott53,"Coldspring, TX",NaN,@SenTedCruz Hold Facebook accountable for abus...,NaN,NaN,NaN,<usernameremoved> Hold Facebook accountable fo...,United States
1,6,2018-04-01,id-980481012550524928,Sun Apr 01 16:24:24 +0000 2018,RShaffer1,"Boston, MA",NaN,Both sides should just drop it. Trump won...el...,NaN,NaN,NaN,Both sides should just drop it. Trump won...el...,United States
2,9,2018-04-01,id-980480980191404032,Sun Apr 01 16:24:16 +0000 2018,pcampisi14,"Toronto, Ontario",NaN,$AAPL lobbing threes while they can. Wasnt it ...,$AAPL lobbing threes while they can. Wasnt it ...,NaN,NaN,$AAPL lobbing threes while they can. Wasnt it ...,Canada
3,16,2018-04-01,id-980481205492682753,Sun Apr 01 16:25:10 +0000 2018,CoinSignalBot,"Seattle, WA",NaN,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,NaN,NaN,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,United States
4,20,2018-04-01,id-980481164334063616,Sun Apr 01 16:25:00 +0000 2018,kimbee802009,"South Dakota, USA",NaN,@SenJohnThune Hold Facebook accountable for ab...,NaN,NaN,NaN,<usernameremoved> Hold Facebook accountable fo...,United States


## Tokenize Sentences and Clean

Removing the emails, new line characters, single quotes and finally split the sentence into a list of words using gensim’s simple_preprocess(). Setting the deacc=True option removes punctuations.

In [11]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  



In [12]:
# Convert to list
data_1 = df_1.texto_completo.values.tolist()
data_words_1 = list(sent_to_words(data_1))
print(data_words_1[:1])

data_2 = df_2.texto_completo.values.tolist()
data_words_2 = list(sent_to_words(data_2))
print(data_words_2[:2])


[['meet', 'our', 'newest', 'product', 'the', 'transparent', 'camhatch', 'webcam', 'cover', 'weve', 'combined', 'everything', 'you', 'loved', 'about', 'the', 'matte', 'black', 'version', 'with', 'the', 'transparency', 'of', 'mark', 'zuckerberg', 'protect', 'your', 'privacy', 'without', 'having', 'to', 'change', 'the', 'appearance', 'of', 'your', 'laptop', 'tablet', 'or', 'smartphone', 'link', 'removed']]
[['usernameremoved', 'hold', 'facebook', 'accountable', 'for', 'abusing', 'our', 'data', 'and', 'censoring', 'our', 'voice', 'vote', 'no', 'on', 'the', 'cra', 'link', 'removed'], ['both', 'sides', 'should', 'just', 'drop', 'it', 'trump', 'won', 'elections', 'over', 'move', 'on', 'link', 'removed']]


## 4. Build the Bigram, Trigram Models and Lemmatize


Let’s form the bigram and trigrams using the Phrases model. This is passed to Phraser() for efficiency in speed of execution.

Next, lemmatize each word to its root form, keeping only nouns, adjectives, verbs and adverbs.

We keep only these POS tags because they are the ones contributing the most to the meaning of the sentences. Here, I use spacy for lemmatization.

In [13]:
#In case you haven't installed yet
#!python -m spacy download en_core_web_sm

In [14]:
# Build the bigram and trigram models
bigram_1 = gensim.models.Phrases(data_words_1, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_1 = gensim.models.Phrases(bigram_1[data_words_1], threshold=100)  
bigram_mod_1 = gensim.models.phrases.Phraser(bigram_1)
trigram_mod_1 = gensim.models.phrases.Phraser(trigram_1)

bigram_2 = gensim.models.Phrases(data_words_2, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_2 = gensim.models.Phrases(bigram_2[data_words_2], threshold=100)  
bigram_mod_2 = gensim.models.phrases.Phraser(bigram_2)
trigram_mod_2 = gensim.models.phrases.Phraser(trigram_2)

In [15]:

# !python3 -m spacy download en  # run in terminal once
"""Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
def process_words(texts, bigram_mod, trigram_mod, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):    
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []    
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out
    



In [16]:
data_ready_1 = process_words(data_words_1, bigram_mod_1, trigram_mod_1)  # processed Text Data!
data_ready_2 = process_words(data_words_2, bigram_mod_2, trigram_mod_2)  # processed Text Data!

KeyboardInterrupt: 

# Recover topic models

In [ ]:
import gensim
import pickle
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary



In [ ]:
route_europe = "C:\\Users\\gonza\\Desktop\\Material tesis\\data\\cambridge_analytica\\regional_datasets\\files_europe\\"
route_northamerica = "C:\\Users\\gonza\\Desktop\\Material tesis\\data\\cambridge_analytica\\regional_datasets\\files_northamerica\\"

In [ ]:
lda_model_1 = LdaModel.load(route_europe+"english_europe_tweets_20190411.csv_gensim.model")
lda_model_2 = LdaModel.load(route_northamerica+"english_northamerica_tweets_20190411.csv_gensim.model")

#### Recover matrix for topic splitting operation

In [20]:
lda_model_1.print_topics()

[(0,
  '0.055*"people" + 0.035*"delete" + 0.032*"year" + 0.026*"account" + 0.022*"friend" + 0.021*"time" + 0.017*"back" + 0.014*"message" + 0.014*"deletefacebook" + 0.014*"call"'),
 (1,
  '0.035*"trump" + 0.033*"leave" + 0.025*"vote" + 0.023*"campaign" + 0.023*"election" + 0.019*"amp" + 0.017*"lie" + 0.015*"government" + 0.014*"democracy" + 0.013*"tory"'),
 (2,
  '0.135*"datum" + 0.104*"privacy" + 0.074*"user" + 0.024*"personal" + 0.022*"information" + 0.021*"law" + 0.019*"access" + 0.018*"app" + 0.017*"give" + 0.015*"sell"'),
 (3,
  '0.085*"datum" + 0.047*"big" + 0.045*"business" + 0.017*"service" + 0.016*"security" + 0.015*"problem" + 0.011*"model" + 0.011*"build" + 0.011*"customer" + 0.010*"online"'),
 (4,
  '0.064*"facebook" + 0.034*"page" + 0.033*"follow" + 0.032*"find" + 0.030*"twitter" + 0.027*"post" + 0.026*"share" + 0.024*"live" + 0.023*"check" + 0.019*"video"'),
 (5,
  '0.045*"question" + 0.034*"today" + 0.026*"watch" + 0.024*"time" + 0.022*"talk" + 0.021*"week" + 0.020*"answ

In [21]:
lda_model_2.print_topics()

[(0,
  '0.070*"facebook" + 0.041*"delete" + 0.036*"year" + 0.032*"post" + 0.031*"twitter" + 0.031*"account" + 0.030*"friend" + 0.025*"page" + 0.023*"follow" + 0.023*"check"'),
 (1,
  '0.031*"time" + 0.024*"back" + 0.018*"put" + 0.015*"man" + 0.014*"show" + 0.014*"start" + 0.013*"turn" + 0.012*"guy" + 0.011*"bring" + 0.011*"play"'),
 (2,
  '0.069*"bigdata" + 0.030*"business" + 0.028*"big" + 0.022*"analytic" + 0.019*"learn" + 0.019*"tech" + 0.018*"ai" + 0.016*"machinelearne" + 0.016*"technology" + 0.014*"marketing"'),
 (3,
  '0.035*"amp" + 0.021*"leave" + 0.020*"love" + 0.019*"free" + 0.019*"conservative" + 0.017*"call" + 0.015*"group" + 0.015*"hate" + 0.015*"stop" + 0.011*"woman"'),
 (4,
  '0.181*"datum" + 0.073*"user" + 0.044*"give" + 0.039*"company" + 0.034*"data" + 0.028*"information" + 0.027*"sell" + 0.027*"share" + 0.025*"personal" + 0.019*"access"'),
 (5,
  '0.082*"make" + 0.047*"good" + 0.038*"thing" + 0.030*"read" + 0.025*"bad" + 0.024*"great" + 0.024*"time" + 0.017*"feel" + 0.0

In [22]:

id2word_1 = Dictionary.load(route_europe+"english_europe_tweets_20190411.csv_id2word")
id2word_2 = Dictionary.load(route_northamerica+"english_northamerica_tweets_20190411.csv_id2word")


In [23]:
corpus_1 = pickle.load(open(route_europe+"english_europe_tweets_20190411.csv_corpus.pkl", 'rb'))
corpus_2 = pickle.load(open(route_northamerica+"english_northamerica_tweets_20190411.csv_corpus.pkl", 'rb'))


<ipython-input-23-64ef41e7a392>:1: ResourceWarning: unclosed file <_io.BufferedReader name='C:\\Users\\gonza\\Desktop\\Material tesis\\data\\cambridge_analytica\\regional_datasets\\files_europe\\english_europe_tweets_20190411.csv_corpus.pkl'>
  corpus_1 = pickle.load(open(route_europe+"english_europe_tweets_20190411.csv_corpus.pkl", 'rb'))
<ipython-input-23-64ef41e7a392>:2: ResourceWarning: unclosed file <_io.BufferedReader name='C:\\Users\\gonza\\Desktop\\Material tesis\\data\\cambridge_analytica\\regional_datasets\\files_northamerica\\english_northamerica_tweets_20190411.csv_corpus.pkl'>
  corpus_2 = pickle.load(open(route_northamerica+"english_northamerica_tweets_20190411.csv_corpus.pkl", 'rb'))


## Get most relevant documents - Europe dataset

In [24]:
#should I use text or text_completo column? in the first, username and link are not removed

In [25]:
'''
See the discusion here:
https://stackoverflow.com/questions/23509699/understanding-lda-transformed-corpus-in-gensim/37708396?noredirect=1#comment77429460_37708396
https://stackoverflow.com/questions/45310925/how-to-get-a-complete-topic-distribution-for-a-document-using-gensim-lda
'''

'\nSee the discusion here:\nhttps://stackoverflow.com/questions/23509699/understanding-lda-transformed-corpus-in-gensim/37708396?noredirect=1#comment77429460_37708396\nhttps://stackoverflow.com/questions/45310925/how-to-get-a-complete-topic-distribution-for-a-document-using-gensim-lda\n'

In [26]:
#with this code we get the full matrix of topic-documents contribution
matrix_documents_topic_contribution_1, _ = lda_model_1.inference(corpus_1)
matrix_documents_topic_contribution_1 /= matrix_documents_topic_contribution_1.sum(axis=1)[:, None]

In [27]:
matrix_documents_topic_contribution_1 = pd.DataFrame(matrix_documents_topic_contribution_1)
matrix_documents_topic_contribution_1.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.067683,0.075224,0.111145,0.083138,0.087522,0.095556,0.066845,0.120091,0.133339,0.066889,0.092569
1,0.095597,0.098896,0.078370,0.078370,0.078370,0.112853,0.078630,0.078384,0.125669,0.079250,0.095611
2,0.087413,0.087413,0.087413,0.087643,0.087413,0.087413,0.087413,0.106643,0.106413,0.087413,0.087413
3,0.083831,0.092424,0.136948,0.097352,0.075758,0.109086,0.077989,0.089814,0.075758,0.076336,0.084705
4,0.081169,0.148244,0.096213,0.093189,0.081169,0.081169,0.093545,0.081761,0.081204,0.081169,0.081169


In [28]:
contents_1 = pd.Series(df_1['text']).reset_index(drop=True)

In [29]:
matrix_documents_topic_contribution_1 = pd.concat([matrix_documents_topic_contribution_1, contents_1], axis=1)


In [30]:
matrix_documents_topic_contribution_1.head()

,0,1,2,3,4,5,6,7,8,9,10,text
0,0.067683,0.075224,0.111145,0.083138,0.087522,0.095556,0.066845,0.120091,0.133339,0.066889,0.092569,Meet our newest product: the transparent CamHa...
1,0.095597,0.098896,0.078370,0.078370,0.078370,0.112853,0.078630,0.078384,0.125669,0.079250,0.095611,@SupportOurLefty This week has been the most #...
2,0.087413,0.087413,0.087413,0.087643,0.087413,0.087413,0.087413,0.106643,0.106413,0.087413,0.087413,Pulitzer prize for this lady. Investigative jo...
3,0.083831,0.092424,0.136948,0.097352,0.075758,0.109086,0.077989,0.089814,0.075758,0.076336,0.084705,"Theres no more secrecy, confident and privacy ..."
4,0.081169,0.148244,0.096213,0.093189,0.081169,0.081169,0.093545,0.081761,0.081204,0.081169,0.081169,AggregateIQ: the obscure Canadian tech firm an...


## Get most relevant documents - North america dataset

In [31]:
#with this code we get the full matrix of topic-documents contribution
matrix_documents_topic_contribution_2, _ = lda_model_2.inference(corpus_2)
matrix_documents_topic_contribution_2 /= matrix_documents_topic_contribution_2.sum(axis=1)[:, None]

In [32]:
matrix_documents_topic_contribution_2 = pd.DataFrame(matrix_documents_topic_contribution_2)
matrix_documents_topic_contribution_2.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.079757,0.079745,0.079745,0.132669,0.097288,0.079757,0.079757,0.079745,0.097289,0.114503,0.079745
1,0.082645,0.100826,0.082645,0.098908,0.082645,0.095298,0.088173,0.082645,0.102745,0.100826,0.082645
2,0.110938,0.093992,0.077042,0.087434,0.110940,0.077284,0.084232,0.093113,0.093991,0.077042,0.093993
3,0.085763,0.085763,0.142317,0.085763,0.085763,0.085763,0.085763,0.085763,0.085763,0.085813,0.085763
4,0.079757,0.079745,0.079745,0.132669,0.097288,0.079757,0.079757,0.079745,0.097289,0.114503,0.079745


In [33]:
contents_2 = pd.Series(df_2['text']).reset_index(drop=True)

In [34]:
matrix_documents_topic_contribution_2 = pd.concat([matrix_documents_topic_contribution_2, contents_2], axis=1)


In [35]:
matrix_documents_topic_contribution_2.head()

,0,1,2,3,4,5,6,7,8,9,10,text
0,0.079757,0.079745,0.079745,0.132669,0.097288,0.079757,0.079757,0.079745,0.097289,0.114503,0.079745,@SenTedCruz Hold Facebook accountable for abus...
1,0.082645,0.100826,0.082645,0.098908,0.082645,0.095298,0.088173,0.082645,0.102745,0.100826,0.082645,Both sides should just drop it. Trump won...el...
2,0.110938,0.093992,0.077042,0.087434,0.110940,0.077284,0.084232,0.093113,0.093991,0.077042,0.093993,$AAPL lobbing threes while they can. Wasnt it ...
3,0.085763,0.085763,0.142317,0.085763,0.085763,0.085763,0.085763,0.085763,0.085763,0.085813,0.085763,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...
4,0.079757,0.079745,0.079745,0.132669,0.097288,0.079757,0.079757,0.079745,0.097289,0.114503,0.079745,@SenJohnThune Hold Facebook accountable for ab...


# Topic similarity metric

In [36]:
# Choose the # top keywords and # top documents a considerar en la metrica

topn_terms = 20
topk_documents = 20
relevance_lambda = 0.6 

ruta_word_embedding = 'data/embedding_english_europe_northamerica_word2vec_300dimensions_cbow_trim3_epoch50.model'
word_embedding_model = gensim.models.Word2Vec.load(ruta_word_embedding)

In [37]:
import topicvisexplorer
import importlib
importlib.reload(topicvisexplorer)
warnings.filterwarnings('ignore')
vis = topicvisexplorer.TopicVisExplorer("borrar_nombre")
topic_similarity_matrix_multicorpora = vis.calculate_topic_similarity_on_multi_corpora(word_embedding_model, lda_model_1,lda_model_2, corpus_1,corpus_2, id2word_1,id2word_2, matrix_documents_topic_contribution_1,matrix_documents_topic_contribution_2, topn_terms, topk_documents, relevance_lambda)

Using Prepared Data script - sort_topics False
Using Prepared Data script - sort_topics False
Calculating for omega =  0.0
Calculating for omega =  0.01
Calculating for omega =  0.02
Calculating for omega =  0.03
Calculating for omega =  0.04
Calculating for omega =  0.05
Calculating for omega =  0.06
Calculating for omega =  0.07
Calculating for omega =  0.08
Calculating for omega =  0.09
Calculating for omega =  0.1
Calculating for omega =  0.11
Calculating for omega =  0.12
Calculating for omega =  0.13
Calculating for omega =  0.14
Calculating for omega =  0.15
Calculating for omega =  0.16
Calculating for omega =  0.17
Calculating for omega =  0.18
Calculating for omega =  0.19
Calculating for omega =  0.2
Calculating for omega =  0.21
Calculating for omega =  0.22
Calculating for omega =  0.23
Calculating for omega =  0.24
Calculating for omega =  0.25
Calculating for omega =  0.26
Calculating for omega =  0.27
Calculating for omega =  0.28
Calculating for omega =  0.29
Calculati

# Show visualization - Multicorpora


In [38]:
#import topicvisexplorer
#import importlib
importlib.reload(topicvisexplorer)

vis = topicvisexplorer.TopicVisExplorer("borrar_nombre")
vis.prepare_multi_corpora( lda_model_1,lda_model_2, corpus_1, corpus_2, id2word_1,id2word_2,  matrix_documents_topic_contribution_1, matrix_documents_topic_contribution_2, topic_similarity_matrix_multicorpora)

Using Prepared Data script - sort_topics False
Using Prepared Data script - sort_topics False


In [39]:
#save data
vis.save_multi_corpora_data("multi_corpora_data_europe_northamerica_ca_lda_mallet_gensim_new_prepared_data_enero_11.pkl")

Multi corpora data saved sucessfully


In [42]:
import topicvisexplorer
import importlib
importlib.reload(topicvisexplorer)

vis = topicvisexplorer.TopicVisExplorer("borrar_nombre")
vis.load_multi_corpora_data("multi_corpora_data_europe_northamerica_ca_lda_mallet_gensim_new_prepared_data_enero_11.pkl")


Data loaded sucessfully


In [44]:
vis.run()

 * Serving Flask app "borrar_nombre" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2021-01-04 14:34:59,271 : INFO :  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


que le pase a jinja  <class 'list'>


2021-01-04 14:35:22,858 : ERROR : Exception on /multicorpora [GET]
Traceback (most recent call last):
  File "c:\users\gonza\topicvisexplorerenv\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\gonza\topicvisexplorerenv\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\gonza\topicvisexplorerenv\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\users\gonza\topicvisexplorerenv\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\users\gonza\topicvisexplorerenv\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\users\gonza\topicvisexplorerenv\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "c:\users\gonza\t

# Show visualization - Single corpora - Europe


In [40]:
#calculate topic similarity metric for one single corpus
# Choose the # top keywords and # top documents a considerar en la metrica
import gensim
topn_terms = 20
topk_documents = 20
relevance_lambda = 0.6 

ruta_word_embedding = 'data/embedding_english_europe_northamerica_word2vec_300dimensions_cbow_trim3_epoch50.model'
word_embedding_model = gensim.models.Word2Vec.load(ruta_word_embedding)

In [41]:
import topicvisexplorer
import importlib
import warnings

importlib.reload(topicvisexplorer)
warnings.filterwarnings('ignore')
vis = topicvisexplorer.TopicVisExplorer("borrar_nombre")

topic_similarity_matrix_1 = vis.calculate_topic_similarity_on_single_corpus(word_embedding_model, lda_model_1, corpus_1, id2word_1, matrix_documents_topic_contribution_1,topn_terms, topk_documents, relevance_lambda)

Using Prepared Data script - sort_topics False
Calculating for omega =  0.0
Calculating for omega =  0.01
Calculating for omega =  0.02
Calculating for omega =  0.03
Calculating for omega =  0.04
Calculating for omega =  0.05
Calculating for omega =  0.06
Calculating for omega =  0.07
Calculating for omega =  0.08
Calculating for omega =  0.09
Calculating for omega =  0.1
Calculating for omega =  0.11
Calculating for omega =  0.12
Calculating for omega =  0.13
Calculating for omega =  0.14
Calculating for omega =  0.15
Calculating for omega =  0.16
Calculating for omega =  0.17
Calculating for omega =  0.18
Calculating for omega =  0.19
Calculating for omega =  0.2
Calculating for omega =  0.21
Calculating for omega =  0.22
Calculating for omega =  0.23
Calculating for omega =  0.24
Calculating for omega =  0.25
Calculating for omega =  0.26
Calculating for omega =  0.27
Calculating for omega =  0.28
Calculating for omega =  0.29
Calculating for omega =  0.3
Calculating for omega =  0.

In [42]:
vis.prepare_single_corpus( lda_model_1, corpus_1, id2word_1, matrix_documents_topic_contribution_1, topic_similarity_matrix_1)

In [43]:
vis.save_single_corpus_data("single_corpus_europe_cambridge_analytica_lda_mallet_gensim_new_prepared_data_enero_11.pkl")


Single corpus data saved sucessfully


# Show visualization - Single corpora - Northamerica


In [44]:
#calculate topic similarity metric for one single corpus
# Choose the # top keywords and # top documents a considerar en la metrica
import gensim
topn_terms = 20
topk_documents = 20
relevance_lambda = 0.6 

ruta_word_embedding = 'data/embedding_english_europe_northamerica_word2vec_300dimensions_cbow_trim3_epoch50.model'
word_embedding_model = gensim.models.Word2Vec.load(ruta_word_embedding)

In [45]:
import topicvisexplorer
import importlib
import warnings

importlib.reload(topicvisexplorer)
warnings.filterwarnings('ignore')
vis = topicvisexplorer.TopicVisExplorer("borrar_nombre")

topic_similarity_matrix_2 = vis.calculate_topic_similarity_on_single_corpus(word_embedding_model, lda_model_2, corpus_2, id2word_2, matrix_documents_topic_contribution_2,topn_terms, topk_documents, relevance_lambda)

Using Prepared Data script - sort_topics False
Calculating for omega =  0.0
Calculating for omega =  0.01
Calculating for omega =  0.02
Calculating for omega =  0.03
Calculating for omega =  0.04
Calculating for omega =  0.05
Calculating for omega =  0.06
Calculating for omega =  0.07
Calculating for omega =  0.08
Calculating for omega =  0.09
Calculating for omega =  0.1
Calculating for omega =  0.11
Calculating for omega =  0.12
Calculating for omega =  0.13
Calculating for omega =  0.14
Calculating for omega =  0.15
Calculating for omega =  0.16
Calculating for omega =  0.17
Calculating for omega =  0.18
Calculating for omega =  0.19
Calculating for omega =  0.2
Calculating for omega =  0.21
Calculating for omega =  0.22
Calculating for omega =  0.23
Calculating for omega =  0.24
Calculating for omega =  0.25
Calculating for omega =  0.26
Calculating for omega =  0.27
Calculating for omega =  0.28
Calculating for omega =  0.29
Calculating for omega =  0.3
Calculating for omega =  0.

In [46]:
vis.prepare_single_corpus( lda_model_2, corpus_2, id2word_2, matrix_documents_topic_contribution_2, topic_similarity_matrix_2)

In [47]:
vis.save_single_corpus_data("single_corpus_northamerica_cambridge_analytica_lda_mallet_gensim_new_prepared_data_enero_11.pkl")

Single corpus data saved sucessfully
